# Importing Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

# import my tANS function
from Functions import Coder, Utils

In [2]:
range_ = (1,7)

# Importing the data

In [3]:
# importing the data
data = [np.load(f"trace/mobilenet_v3/input_{i}.npy") for i in range(range_[0],range_[1])]
#data_flat = data.flatten()
np.unique(data[0]), data[0].shape

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

# Compressing the data


1. Convert each data point to a *(symbol, offset)* pair
2. Compress Values to value stream
3. Put offset in offset stream

### Get frequencies

In [4]:
# Calculate frequency of each uint8 value
def calculate_frequency(array):
    # Ensure the input array is of type uint8
    if array.dtype != np.uint8:
        raise ValueError("Input array must be of type uint8")
    
    # Initialize an array of zeros with a length of 256 to store frequencies
    frequency = np.zeros(256, dtype=int)
    
    # Iterate through the array and count the occurrences of each value
    for value in array:
        frequency[value] += 1
        
    return frequency

In [5]:
freqs = [calculate_frequency(d) for d in data]

# rescale the frequencies to a power of 2
freqs = [Utils.rescale_list_to_power_of_2(freq, 2**10) for freq in freqs]
print(sum(freqs[0]))

1024


In [6]:
data[0][0].shape

(3, 224, 224)

In [7]:
import time
nbits = 8 # takes 4 bits to represent each symbol

all_run_times = []
all_build_times = []
all_comp_ratios = []
all_bps = []

for i in range(len(freqs)):
    print(f"Compressing Layer {i}")

    run_times = []
    build_times = []
    comp_ratios = []
    bp_sym = []

    for j in range(len(data[i])):
        print(f"\tCompressing tensor {j}")
        
        time_start = time.time()
        
        c = Coder.Coder(sum(freqs[i]), [i for i in range(len(freqs[i]))], freqs[i], fast = False)
        
        time_end = time.time()
        build_time_taken = time_end - time_start

        msg = list(data[i][j].flatten())

        time_start = time.time()
        out, comp_bits = c.encode_decode(msg)
        time_end = time.time()
        run_time_taken = time_end - time_start

        if out != msg:
            print("Error in encoding and decoding")
            break
        
        run_times.append(run_time_taken)
        build_times.append(build_time_taken)
        comp_ratios.append(len(msg) * nbits / comp_bits)
        bp_sym.append(comp_bits / len(msg))
        
    # print average stats
    print("\tAverage run time taken: %f seconds" % np.mean(run_times))
    print("\tAverage build time taken: %f seconds" % np.mean(build_times))
    print("\tAverage compression ratio: %f" % np.mean(comp_ratios))
    print("\tAverage bits per symbol: %f" % np.mean(bp_sym))
    
    # add stats to all lists
    all_run_times.append(run_times)
    all_build_times.append(build_times)
    all_comp_ratios.append(comp_ratios)
    all_bps.append(bp_sym)

Compressing Layer 0
	Compressing tensor 0
	Compressing tensor 1
	Compressing tensor 2
	Compressing tensor 3
	Compressing tensor 4
	Compressing tensor 5
	Compressing tensor 6
	Compressing tensor 7
	Compressing tensor 8
	Average run time taken: 208.059594 seconds
	Average build time taken: 0.348622 seconds
	Average compression ratio: 1070439.111111
	Average bits per symbol: 0.000007
Compressing Layer 1
	Compressing tensor 0
	Compressing tensor 1
	Compressing tensor 2
	Compressing tensor 3
	Compressing tensor 4
	Compressing tensor 5
	Compressing tensor 6
	Compressing tensor 7
	Compressing tensor 8
	Average run time taken: 207.221485 seconds
	Average build time taken: 0.353526 seconds
	Average compression ratio: 1070439.111111
	Average bits per symbol: 0.000007
Compressing Layer 2
	Compressing tensor 0
	Compressing tensor 1
	Compressing tensor 2


In [ ]:
# display stats in a dataframe
import pandas as pd

freqs = freqs

stats_1 = pd.DataFrame({"Layer": [i for i in range(len(freqs))],
                      "Run Time": [np.mean(all_run_times[i]) for i in range(len(freqs))],
                      "Build Time": [np.mean(all_build_times[i]) for i in range(len(freqs))],
                      "Compression Ratio": [np.mean(all_comp_ratios[i]) for i in range(len(freqs))],
                      "Bits per Symbol": [np.mean(all_bps[i]) for i in range(len(freqs))]})


KeyboardInterrupt



In [ ]:
stats_1

,Layer,Run Time,Build Time,Compression Ratio,Bits per Symbol
0,0,1.865475,0.011845,4.192465,0.954863
1,1,0.007848,0.049538,3.668388,1.090712
2,2,0.007443,0.009071,2.039716,1.963778
3,3,0.007627,0.009255,2.039716,1.963778
